In [ ]:
from utils import *

# Carica lo shapefile e il modello YOLO
shapefile_path = "/lustrehome/cilli/deepforest/YOLOv11_tree_detector/ROI/bat_grid.shp" #"Puglia_Polygon.shp" per estendere la classificazione a tutta la Puglia
gdf = gpd.read_file(shapefile_path)
model = YOLO( "/lustrehome/cilli/deepforest/YOLOv11_tree_detector/ulivi_project/ulivi/weights/best.pt")
model.overrides['max_det'] = 2000
model.overrides['conf'] = 0.5 #Soglia severa per non salvare falsi positivi - Questo conduce a circa 60M di alberi per la Puglia - vicina ai circa 60M pre xylella e 40 M post xylella

# Connessione al servizio WMS
wms_url = "http://webapps.sit.puglia.it/arcgis/services/BaseMaps/Ortofoto2019/ImageServer/WMSServer/service?"
wms = WebMapService(wms_url, version='1.1.1')
layer_name = "Ortofoto2019"
output_dir = "/lustrehome/cilli/deepforest/YOLOv11_tree_detector/ulivi_project/shapes/"



# Esegui il processo
process_all_tiles_in_parallel(gdf, wms, layer_name, model, output_dir)


Ultralytics 8.3.98 🚀 Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA A100-PCIE-40GB, 40326MiB)

Ultralytics 8.3.98 🚀 Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA A100-PCIE-40GB, 40326MiB)

Ultralytics 8.3.98 🚀 Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA A100-PCIE-40GB, 40326MiB)

Ultralytics 8.3.98 🚀 Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA A100-PCIE-40GB, 40326MiB)

Ultralytics 8.3.98 🚀 Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA A100-PCIE-40GB, 40326MiB)

Ultralytics 8.3.98 🚀 Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA A100-PCIE-40GB, 40326MiB)

Ultralytics 8.3.98 🚀 Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA A100-PCIE-40GB, 40326MiB)


Ultralytics 8.3.98 🚀 Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA A100-PCIE-40GB, 40326MiB)
Ultralytics 8.3.98 🚀 Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA A100-PCIE-40GB, 40326MiB)

YOLO11n-seg summary (fused): 113 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs
YOLO11n-seg summary (fused): 113 layers, 2,834,763 param

In [3]:
# Cartella con i shapefile e file di output finale
shapefile_folder = "/lustrehome/cilli/deepforest/YOLOv11_tree_detector/ulivi_project/shapes/"
output_file = "/lustrehome/cilli/deepforest/YOLOv11_tree_detector/ulivi_project/BOXES_BAT.shp"

# Poligono di delimitazione NUTS
nuts_gdf = gpd.read_file("/lustrehome/cilli/deepforest/YOLOv11_tree_detector/ROI/bat.shp")
nuts_gdf.to_crs(epsg=32633)
nuts_boundary = nuts_gdf.unary_union  # Unione delle geometrie per il filtro


# Ottieni la lista di tutti i file .shp nella cartella
all_shapefiles = [os.path.join(shapefile_folder, f) for f in os.listdir(shapefile_folder) if f.endswith(".shp")]

# Esegui il caricamento e il filtraggio parallelo
gdfs = Parallel(n_jobs=-1)(delayed(read_and_filter_shapefile)(shp) for shp in tqdm(all_shapefiles))

# Rimuovi eventuali None (shapefile senza geometrie valide)
gdfs = [gdf for gdf in gdfs if gdf is not None]

# Unisci tutti i GeoDataFrame
merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Qui il risultato ottenuto usando layer LPIS come filtro, per una mappa più accurata 
# Salva il file unito
#merged_gdf.to_file(output_file)


/tmp/ipykernel_96632/2090281335.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  nuts_boundary = nuts_gdf.unary_union  # Unione delle geometrie per il filtro
100%|██████████| 33380/33380 [02:47<00:00, 199.87it/s]


In [5]:
merged_gdf.to_file(output_file)
